In [1]:
import nltk
import pandas as pd
import re
import regex
import string
from sklearn.feature_extraction.text import CountVectorizer
from nltk.stem.porter import PorterStemmer
from nltk.stem.lancaster import LancasterStemmer
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.decomposition import NMF


In [2]:
df=pd.read_csv('reddit.csv')

### EDA & Data Cleaning 

In [3]:
df.head()
df.drop('Unnamed: 0',axis=1,inplace=True)


In [4]:
#remove empty rows
df.dropna(inplace=True)
df.reset_index(inplace=True)

In [5]:
#check for duplicates 
df[df.duplicated()]

,index,title,post


In [ ]:
#nba['post']=nba['post'].str.replace('^https?:\/\/[\da-z\.-]+\.[a-z\.]{2,6}[^\s]+', '',regex = True)

In [7]:
#URL link removal
removeUrl = lambda x: re.sub('https?:\/\/[\da-z\.-]+\.[a-z\.]{2,6}[^\s]+', "", x)
df['post']=df.post.map(removeUrl)

In [8]:
#removing numbers -- captial letters and punctuation

remove_num = lambda x: re.sub('\w*\d\w*', ' ', x)
punc_lower = lambda x: re.sub('[%s]' % re.escape(string.punctuation), ' ', x.lower())

df['post']=df.post.map(remove_num).map(punc_lower)

In [9]:
# toknize
df['post']=df.post.map(word_tokenize)

In [10]:
#stemming 
#st = LancasterStemmer()
#def stemming(text):
    #stem_text = [st.stem(word) for word in text]
    #return stem_text
#nba['post']= nba['post'].apply(lambda x: stemming(x))
 
def listToString(s): 
    
    # initialize an empty string
    str1 = " " 
    
    # return string  
    return (str1.join(s))

In [11]:
#defining the function for lemmatization
wordnet_lemmatizer = WordNetLemmatizer()
def lemmatizer(text):
    lemm_text = [wordnet_lemmatizer.lemmatize(word) for word in text]
    return lemm_text
df['post']=df['post'].apply(lambda x:lemmatizer(x))
df['post']=df.post.map(listToString)

In [13]:
cv1 = CountVectorizer(stop_words='english')
c= cv1.fit_transform(df.post)

In [14]:
pd.DataFrame(c.toarray(), columns=cv1.get_feature_names()).head()
#pd.DataFrame(c.toarray(),index=nba['post'], columns=cv1.get_feature_names()).head()

,aaa,aaron,aau,ab,aba,abandon,abandoned,abandoning,abdel,abducted,...,zydrunas,álvaro,ďays,šarić,štrba,сhina,细拉,阿姆西,震旦与食人魔实机兵种展示,𝘳𝘦𝘢𝘭𝘭𝘺
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
cv1.get_feature_names()[2000:3000]

### NMF

In [15]:
nmf_model = NMF(5)
doc_topic = nmf_model.fit_transform(c)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\decomposition\_nmf.py:312: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  warnings.warn(("The 'init' value, when 'init=None' and "


In [16]:
topic_word = pd.DataFrame(nmf_model.components_.round(3),
             index = ["component_1","component_2",'component_3','component_4','component_5'],
             columns = cv1.get_feature_names())
topic_word

,aaa,aaron,aau,ab,aba,abandon,abandoned,abandoning,abdel,abducted,...,zydrunas,álvaro,ďays,šarić,štrba,сhina,细拉,阿姆西,震旦与食人魔实机兵种展示,𝘳𝘦𝘢𝘭𝘭𝘺
component_1,0.052,0.000,0.000,0.0,0.000,0.002,0.000,0.002,0.000,0.025,...,0.001,0.000,0.0,0.000,0.027,0.000,0.006,0.000,0.000,0.005
component_2,0.000,0.004,0.294,0.0,0.001,0.000,0.000,0.001,0.000,0.001,...,0.008,0.000,0.0,0.005,0.000,0.003,0.001,0.000,0.000,0.001
component_3,0.014,0.000,0.000,0.0,0.001,0.000,0.001,0.002,0.000,0.005,...,0.001,0.000,0.0,0.000,0.000,0.001,0.000,0.000,0.000,0.000
component_4,0.002,0.319,0.000,0.0,0.001,0.000,0.000,0.000,0.047,0.001,...,0.001,0.000,0.0,0.000,0.000,0.001,0.000,0.000,0.000,0.001
component_5,0.096,0.000,0.000,0.0,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.089,0.0,0.000,0.006,0.000,0.001,0.001,0.001,0.000


In [17]:
def display_topics(model, feature_names, no_top_words, topic_names=None):
    for ix, topic in enumerate(model.components_):
        if not topic_names or not topic_names[ix]:
            print("\nTopic ", ix)
        else:
            print("\nTopic: '",topic_names[ix],"'")
        print(", ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))

In [18]:
display_topics(nmf_model, cv1.get_feature_names(), 20)


Topic  0
game, horizon, forza, series, xsx, racing, best, party, like, mario, new, ha, open, time, pc, critic, xbox, star, superstar, make

Topic  1
team, player, nba, jvbl, league, year, school, new, basketball, season, contract, organization, like, play, just, think, make, game, time, deal

Topic  2
game, world, banjo, character, wa, duo, tooie, way, ha, just, transformation, original, like, make, kazooie, mumbo, fight, pad, ability, lot

Topic  3
team, pf, pt, nbsp, nba, ast, stl, blk, pm, game, com, min, fg, stats, ft, pg, sf, sg, reb, fgm

Topic  4
tensei, megami, shin, game, nintendo, switch, combat, jrpg, fan, ha, series, star, time, atlus, entry, make, new, trailer, like, feel


In [21]:
#df.to_csv('check.csv')